In [25]:
import pandas as pd
import requests
import json
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [26]:
def formatted_addr(addr):
    addr_key = '1f206129ec240f1cc12be917b9615c0b'
    _url = 'https://restapi.amap.com/v3/geocode/geo?address={}&output=JSON&key={}'.format(addr, addr_key)
    respon = requests.get(_url)
    response = json.loads(respon.content)
    status = response['status']
    info = response['info']
    addr_data = '' + "&" + '' + "&" + '' + "&" + '' + "&" + '' + "&" + '0.0,0.0'
    if status == '1' and info == 'OK':
        count = response['count']
        if count == '1':
            data = response["geocodes"][0]
            formatted_address = data["formatted_address"]
            country = data["country"]
            province = data["province"]
            city = data["city"]
            district = data["district"]
            location = data["location"]
            addr_data = str(formatted_address) + "&" + str(country) + "&" + str(province) + "&" + str(
                city) + "&" + str(district) + "&" + str(location)
        elif count == '0':
            addr_data = '' + "&" + '' + "&" + '' + "&" + '' + "&" + '' + "&" + '0.0,0.0'
    elif status == '0' and info != 'OK':
        addr_data = '' + "&" + '' + "&" + '' + "&" + '' + "&" + '' + "&" + '0.0,0.0'

    r0 = addr_data.split('&')[0] # address
    r1 = addr_data.split('&')[1] # country
    r2 = addr_data.split('&')[2] # province
    r3 = addr_data.split('&')[3] # city
    r4 = addr_data.split('&')[4] # district
    r5 = addr_data.split('&')[5] # Latitude and longitude
    print(r0)
    if r3 == '' or r3 == "[]":
        r3 = addr.strip()
    return r3
# formatted_addr('Beijing and SHENZHEN')
gd_udf = F.udf(formatted_addr,T.StringType())

In [27]:
# todo 1： 读取数据
read_path = 'abfss://data-warehouse-ods@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/ods_mau_bcd_aviation_city_code.csv'
df = pd.read_csv(read_path,dtype=str)

# todo 2：数据处理
new_col = {
    '三字代码'  : 'aviation_city_code'
    ,'所属城市'  : 'city_name_local_language'
    ,'英文名称'  : 'city_name_EN'
    ,'所属国家'  : 'country_region_name'
    ,'国家代码'  : 'country_region_code'
}
df.rename(columns=new_col,inplace=True)
df['country_region_name'].replace('西藏','中国',inplace=True)
df['city_name_local_language'] = df['city_name_local_language'].str.strip() # 去除行首空格
df = df.reindex(columns=new_col.values())
df = spark.createDataFrame(df)

df = df.withColumn('city_name_local_language'
                ,F.when(F.col('country_region_name')=='中国',gd_udf(F.col('city_name_local_language'))) \
                .otherwise(F.col('city_name_local_language')))
display(df)


In [28]:
# 写入文件
save_path = 'abfss://data-warehouse-dim@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/dim_aviation_city_code_mf.csv'
df.toPandas().to_csv(save_path, mode='w',index=False, header=True)